<a href="https://colab.research.google.com/github/samirjhb/samirjhb/blob/main/examen_1_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## INSTALACION DE PAQUETE
%%capture
!pip install pyspark
!pip install pyarrow

In [2]:
## CONEXION DRIVE
from google.colab import drive
drive.mount("/content/gdrive")
gdrive='/content/gdrive/MyDrive/big data/'
segmentacion = gdrive + 'Tabla_segmentacion.csv'
pricing = gdrive + 'Tabla_pricing.csv'

Mounted at /content/gdrive


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min as pyspark_min, max as pyspark_max
from pyspark.sql.functions import count, avg, col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc
from pyspark.sql import functions as F


# Crear una sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("Ejemplo").getOrCreate()

# Cargar DataFrames desde archivos CSV
df_seg = spark.read.csv(segmentacion, inferSchema=True, header =True,sep=";")
df_pri = spark.read.csv(pricing, inferSchema=True, header =True,sep=";")

In [15]:
## Punto 1
# Contar registros en cada DataFrame
conteo_seg = df_seg.count()
conteo_pri = df_pri.count()

# Calcular la unión de ambos DataFrames
union_registros = conteo_seg + conteo_pri

# Imprimir resultados
print(f"Total de segmentacion: {conteo_seg}")
print(f"Total de pricing: {conteo_pri}")
print(f"Total en ambos archivos: {union_registros}")

Total de segmentacion: 2805667
Total de pricing: 12581326
Total en ambos archivos: 15386993


In [23]:
## Punto 2
# Seleccionar solo las columnas relevantes
df_pri_select = df_pri.select("FAMILIA", "MARGEN")

# Calcular los rangos mínimos y máximos por familia
resultados_familias = df_pri_select.groupBy("FAMILIA").agg(
    pyspark_min("MARGEN").alias("min_margen"),
    pyspark_max("MARGEN").alias("max_margen")
)

# Mostrar resultados
resultados_familias.show(truncate=False)

+-------+------------------+------------------+
|FAMILIA|min_margen        |max_margen        |
+-------+------------------+------------------+
|12     |0.1306476481437908|0.6862996158770807|
|1      |0.1306476481437908|0.6543916706056292|
|13     |0.1306476481437908|0.5705853658536586|
|6      |0.1306476481437908|0.9984176477444924|
|3      |0.1306476481437908|0.5800000000000001|
|5      |0.1306476481437908|0.9989324223337248|
|9      |0.1306476481437908|0.7539181286549708|
|4      |0.1306476481437908|0.8433924466193962|
|8      |0.1306476481437908|0.9423213066112972|
|7      |0.1306476481437908|0.6961444508475835|
|10     |0.1306476481437908|0.9348571428571428|
|11     |0.1306476481437908|0.6968634975081676|
|2      |0.1306476481437908|0.9129213483146068|
+-------+------------------+------------------+



In [21]:
## Punto 3

# Agrupar y realizar las agregaciones
totalConCliente = df_seg.agg(
    F.countDistinct("RUT_CLI").alias("RUT_CLI"),  # Contar clientes distintos
    F.countDistinct("SEGMENTO").alias("SEGMENTO"),  # Contar segmentos distintos
    (F.countDistinct("RUT_CLI") / F.countDistinct("SEGMENTO")).alias("PROMEDIO_CLIENTES")  # Calcular el promedio
)

# Imprimir resultados
totalConCliente.show()


+-------+--------+-----------------+
|RUT_CLI|SEGMENTO|PROMEDIO_CLIENTES|
+-------+--------+-----------------+
|  13296|    5371|2.475516663563582|
+-------+--------+-----------------+



In [10]:
## Punto 4

# Contar la cantidad de rubros distintos en df_pri
rubros_count = df_pri.agg(countDistinct("RUBRO").alias("NUM_RUBROS"))

# Mostrar resultados
rubros_count.show()


+----------+
|NUM_RUBROS|
+----------+
|        17|
+----------+



In [4]:
## Punto 5

# Encontrar el cliente con el mayor precio
cliente_mayor_precio = df_pri.orderBy(desc("MARGEN")).first()

# Mostrar resultados
print("Cliente con el mayor precio:")
print(cliente_mayor_precio)

Cliente con el mayor precio:
Row(SEGMENTO=1, SUCURSAL='E537', RUBRO='C5 ', FAMILIA=5, SUBFAMILIA=11, PRODUCTO=None, JERARQUIA=2, MARGEN=0.9989324223337248, MARGEN_PRE_APRO=0.9989324223337248)


In [5]:
# Encontrar el cliente con el mayor precio
cliente_mayor_precio = df_pri.orderBy(desc("MARGEN")).first()

# Mostrar resultados
print("Cliente con el mayor precio:")
print(cliente_mayor_precio)

# Obtener el RUT_CLI correspondiente al SEGMENTO del cliente con el mayor precio
segmento_cliente_mayor_precio = cliente_mayor_precio["SEGMENTO"]
# Verificar si el SEGMENTO no es NULL antes de buscar el RUT_CLI
if segmento_cliente_mayor_precio is not None:
    rut_cliente_mayor_precio = df_seg.filter(col("SEGMENTO") == segmento_cliente_mayor_precio).select("RUT_CLI").first()

    # Mostrar el RUT_CLI correspondiente
    print("RUT_CLI del cliente con el mayor precio:")
    print(rut_cliente_mayor_precio["RUT_CLI"])
else:
    print("El SEGMENTO es NULL, no se puede buscar el RUT_CLI correspondiente.")

Cliente con el mayor precio:
Row(SEGMENTO=1, SUCURSAL='E523', RUBRO='C5 ', FAMILIA=5, SUBFAMILIA=11, PRODUCTO=None, JERARQUIA=2, MARGEN=0.9989324223337248, MARGEN_PRE_APRO=0.9989324223337248)
RUT_CLI del cliente con el mayor precio:
None


In [6]:
# Encontrar el cliente con el mayor precio
# Ordenar el DataFrame por el campo "MARGEN" en orden descendente
df_ordenado = df_pri.orderBy(desc("MARGEN"))

# Obtener el segundo cliente con mayor precio (o margen)
cliente_mayor_precio = df_ordenado.take(2)[-1]

# Mostrar resultados
print("Cliente con el mayor precio:")
print(cliente_mayor_precio)

# Obtener el RUT_CLI correspondiente al SEGMENTO del cliente con el mayor precio
segmento_cliente_mayor_precio = cliente_mayor_precio["SEGMENTO"]
# Verificar si el SEGMENTO no es NULL antes de buscar el RUT_CLI
if segmento_cliente_mayor_precio is not None:
    rut_cliente_mayor_precio = df_seg.filter(col("SEGMENTO") == segmento_cliente_mayor_precio).select("RUT_CLI").first()

    # Mostrar el RUT_CLI correspondiente
    print("RUT_CLI del cliente con el mayor precio:")
    print(rut_cliente_mayor_precio["RUT_CLI"])
else:
    print("El SEGMENTO es NULL, no se puede buscar el RUT_CLI correspondiente.")

Cliente con el mayor precio:
Row(SEGMENTO=13079, SUCURSAL='E535', RUBRO='C5 ', FAMILIA=5, SUBFAMILIA=11, PRODUCTO=None, JERARQUIA=2, MARGEN=0.9989324223337248, MARGEN_PRE_APRO=0.9780354404971202)
RUT_CLI del cliente con el mayor precio:
66666666


In [7]:
## Punto 6

# Calcular el margen promedio por sucursal en df_pri
margen_promedio_por_sucursal = df_pri.groupBy("SUCURSAL").agg(avg("MARGEN").alias("MARGEN_PROMEDIO"))

# Encontrar la sucursal con el mayor margen promedio
sucursal_max_margen = margen_promedio_por_sucursal.orderBy("MARGEN_PROMEDIO", ascending=False).first()

# Encontrar la sucursal con el menor margen promedio
sucursal_min_margen = margen_promedio_por_sucursal.orderBy("MARGEN_PROMEDIO").first()

# Mostrar resultados
print("Sucursal con el mayor margen promedio:")
print(sucursal_max_margen)

print("\nSucursal con el menor margen promedio:")
print(sucursal_min_margen)

Sucursal con el mayor margen promedio:
Row(SUCURSAL='E550', MARGEN_PROMEDIO=0.3260823808111714)

Sucursal con el menor margen promedio:
Row(SUCURSAL='E576', MARGEN_PROMEDIO=0.2753196305799647)


In [22]:
# Imprimir el esquema del DataFrame df_pri y df_seg
# df_pri.printSchema()
# df_seg.printSchema()
df_pri.show()
df_seg.show()

+--------+--------+-----+-------+----------+--------+---------+------------------+------------------+
|SEGMENTO|SUCURSAL|RUBRO|FAMILIA|SUBFAMILIA|PRODUCTO|JERARQUIA|            MARGEN|   MARGEN_PRE_APRO|
+--------+--------+-----+-------+----------+--------+---------+------------------+------------------+
|   31820|    E523|  I4 |     13|         2|    NULL|        2|0.3489893009708009|0.2990265662307124|
|   31820|    E523|  I5 |     13|         2|    NULL|        2| 0.374112013931368|0.3388983988296916|
|   31820|    E523|  O4 |     13|         2|    NULL|        2| 0.376833211102885|0.3505669123706935|
|   31821|    E523|  C7 |     13|         2|    NULL|        2|0.3707050035558116| 0.352306428211865|
|   31822|    E523|  I3 |     13|         2|    NULL|        2|0.2661791631184857|0.2626256817124447|
|   31823|    E523|  C6 |     13|         2|    NULL|        2| 0.266063059646467|0.2626256817124447|
|   31823|    E523|  I2 |     13|         2|    NULL|        2|0.2665880721155131|